In [1]:
import ee
import os
import geojson
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp
from geemap import png_to_gif
from geemap.cartoee import get_map, add_gridlines, add_scale_bar_lite, add_north_arrow
ee.Initialize()
%matplotlib inline

In [2]:
filename = "gibraltar.geojson"
with open("Locations\\" + filename) as f:
    gj = geojson.load(f)

coords = gj['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [3]:


start_date = "2021-12-01" # 
end_date = "2021-12-31" #

In [4]:
ffa_db = (ee.ImageCollection('COPERNICUS/S1_GRD')
                       .filterBounds(aoi)
                       .filterDate(ee.Date(start_date), ee.Date(end_date))
                       .sort('system:time_start'))

In [5]:
import time
acq_times = ffa_db.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['12/01/21',
 '12/01/21',
 '12/02/21',
 '12/02/21',
 '12/03/21',
 '12/07/21',
 '12/08/21',
 '12/08/21',
 '12/09/21',
 '12/13/21',
 '12/13/21',
 '12/14/21',
 '12/14/21',
 '12/15/21',
 '12/19/21',
 '12/20/21',
 '12/20/21',
 '12/21/21',
 '12/25/21',
 '12/25/21',
 '12/26/21']

In [22]:
im_list = ffa_db.toList(ffa_db.size())
index = 6
url = ee.Image(im_list.get(index)).select('VV').clip(aoi).getThumbURL({'min': -15, 'max': 10})
disp.Image(url=url, width=1200)

In [23]:
import requests

img_data = requests.get(url).content
with open('Images/gibraltar7.png', 'wb') as handler:
    handler.write(img_data)